In [1]:
# import our usual things
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# import kepler 101 dataset
from hermite_library import read_hermite_solution_from_file

t_h, E_h, r_h, v_h = read_hermite_solution_from_file('myPlanetSystem_kepler101_solution1.txt')

In [4]:
from hermite_library import AUinCM # for converting between AU and cm

# Choosing locations of files

In [5]:
# specify the directory where the make3dplanets.py is 
make3dplanets_libdir = './genericPlanetFiles/'

In [6]:
# pick a name for our 3D files
SystemName = 'myPlanetSystem1'

In [7]:
# make a directory where things are going to be stored
#!mkdir outPlanets
# now I commented because directory exists

In [41]:
output_planet_dir = './outPlanets/' # where my planets will be stored

# Make aesthecial choices

In [42]:
# look again at the kepler dataset to remind ourselves what things look like
from convert_kepler_data import read_kepler_data

In [43]:
kepler_data = read_kepler_data('kepler101data.txt') # initial conditions, observational

In [44]:
# mass of the star here in this system
kepler_data['sMass'][0] # mass of star in solar masses

1.17

In [45]:
# mass of my planets
kepler_data['pMass'] # in Jupiter masses

array([0.16, 0.01], dtype=float32)

In [46]:
# so these are about the size of Saturn & Neptune, respectively

In [47]:
#  order is        first planet       second planet            sun
texture_files = ['saturn_1024.jpg', 'neptunemap_1000.jpg', 'sun_texture1.jpg']

In [64]:
# approximate the sizes of things by Saturn, Neptune and the sun
PlanetRadius = np.array([5.823e9, 2.46e9, 69.55e9])*2 # cm
#               Saturn   Neptune  Sun

In [65]:
PlanetRadius

array([1.1646e+10, 4.9200e+09, 1.3910e+11])

# Start putting all this information together

In [66]:
# 1. tell python where to look for files
from sys import path
path.append(make3dplanets_libdir)
from make3dplanets import make3dplanets

In [67]:
# rename a few things for sanity's sake
generic_dir = make3dplanets_libdir # generic planet files
textures_dir = make3dplanets_libdir + '/textureMaps/' # textures are 

# Finally - run function to make 3d geometry

In [68]:
# what time step to make a file out of
Nplot = 100

In [69]:
# format my planet locations

In [70]:
# at this timestep (Nplot timestep) our planet will be located at PlanetLocation
# start with an empty PlanetLocation vector:
PlanetLocation = np.zeros([len(PlanetRadius), 3]) # initialize format
# fill PlanetLocation with positions from r_h, our simulation data at timestep Nplot
for p in range(0,len(PlanetRadius)): # loop over each planet
    PlanetLocation[p,2] = r_h[p, 0, Nplot]*AUinCM # swapping z & x, and converting to cm
    PlanetLocation[p,0] = r_h[p, 1, Nplot]*AUinCM # swapping x & y, converting
    PlanetLocation[p,1] = r_h[p, 2, Nplot]*AUinCM # swapping y & z, converting


In [71]:
# use all this information to make a 3d geometry file
fname = make3dplanets(SystemName, PlanetLocation, PlanetRadius, 
                     output_planet_dir, generic_dir, textures_dir=textures_dir, 
                     texture_file=texture_files, DistanceUnits=AUinCM)

In [55]:
fname

'myPlanetSystem1.obj'

## Exercise
Do this for your dataset (not the Kepler 101 dataset). Upload a model + textures to sketchfab.

Consider:
1. What textures you're going to use? (What do you think your planet looks like?)
1. Do you want to change the size (PlanetRadius), if so, make sure you note by how much.

Other things:
1. Try other timesteps.
1. Look over the galaxy data
1. Try another model - either seperate file upload OR in same box, offset
1. Start building a data-viz blog

# Using rebound to solve for solar system - Bonus Round

In [97]:
# gotta do:
#!pip install rebound

from rebound import data
import rebound

In [98]:
sim = rebound.Simulation() # create a simulation object

In [99]:
data.add_outer_solar_system(sim) # add the outer solar system data from Horizon telescope
# gives position of outer planets

In [100]:
# counts particles in sim
Nparts = 0
for p in sim.particles:
    Nparts += 1
print('number parts ', Nparts)

number parts  6


In [101]:
# timestep counter
Nsteps = 100 # total number of steps in sim
delta_t = 1.0e2 # time in *days* between steps

# create r_h to plop down our positions in
# r_h[# particles, #of coord, #timesteps]
r_h = np.zeros([Nparts, 3, Nsteps])
#r_h.shape

In [102]:
# Integrate until t=1e4 (unit of time in this example is days)
for i in range(Nsteps):
    tmax = delta_t*i
    sim.integrate(tmax) # integrate to current timestep
    # Store positions in r_h array
    for j,p in enumerate(sim.particles):
        #     time       x    y    z 
        #print(j,sim.t, p.x, p.y, p.z)
        r_h[j, 0, i] = p.x
        r_h[j, 1, i] = p.y
        r_h[j, 2, i] = p.z



In [103]:
# plot quick scatter plot with ipyvolume
import ipyvolume

In [104]:
x = r_h[:,0,:].ravel()
y = r_h[:,1,:].ravel()
z = r_h[:,2,:].ravel()
ipyvolume.quickscatter(x, y, z, 
                       size=1, marker="sphere")